<h3> Extracting protein sequences' features using ProtBert-BFD pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -q transformers
!pip install -q torch
!pip install -q tqdm
!pip install -q wandb
!pip install -q biopython
!pip install matplotlib

In [3]:
!pip install -q transformers

In [4]:
import torch
from transformers import BertModel, BertTokenizer
import re
import os
import requests
from tqdm.auto import tqdm
import wandb
!wandb login 045176213138ac579c64067809395b0ad25a8cf9

# Initialize wandb
wandb.login()
wandb.init(project="prot_bert_bfd_project", name="ProtBERT_Training")

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: julian37 (julian37-vera-biotechnologies). Use `wandb login --relogin` to force relogin


<b>2. Load the vocabulary and ProtBert-BFD Model</b>

In [5]:
tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
model = BertModel.from_pretrained("Rostlab/prot_bert_bfd")

<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
# Log model configuration to wandb
wandb.config.update({
    "model": "ProtBERT-BFD",
    "architecture": "BertModel"
})

# Log the model architecture
wandb.watch(model)

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [9]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [10]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>5. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [11]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
!pip install wandb
!wandb login 045176213138ac579c64067809395b0ad25a8cf9

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
import wandb
import random
import torch
import os
import pandas as pd

# start a new wandb run to track this script
wandb.init(
    project="chemberta_run",  # Customize the project name

    # track hyperparameters and run metadata
    config={
        "learning_rate": 0.02,   # Example hyperparameter
        "architecture": "CNN",   # Replace this if necessary
        "dataset": "ClinTox",  # Replace with the actual dataset used
        "epochs": 1,
    }
)

# Simulate or place actual training loop
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

# inputs and outputs for sequences
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)


# log metrics to wandb during training
wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

acc,▁
loss,▁
acc,0.82903
loss,0.20241


<b>6. Extracting sequences' features and load it into the CPU if needed<b>

In [14]:
# Ensure the model and input tensors are on the same device
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
model.to(device)

# Feature extraction (inference mode)
with torch.no_grad():
    embedding = model(input_ids=input_ids, attention_mask=attention_mask)[0]


In [15]:
embedding = embedding.cpu().numpy()

<b>7. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtBert-BFD model<b>

In [16]:
# Removing padding ([PAD]) and special tokens ([CLS],[SEP])
features = []
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()  # Actual length of the sequence
    seq_emb = embedding[seq_num][1:seq_len-1]  # Remove [CLS] and [SEP]
    features.append(seq_emb)

# Initialize wandb
wandb.init(project="prot_bert_bfd_project", name="ProtBERT_Training")

# Log the cleaned features
for i, seq_feature in enumerate(features):
    wandb.log({
        f"sequence_{i}_cleaned_features": seq_feature,
    })

# You can also save the features as a file and log the file in wandb:
torch.save(features, '/content/protbert_cleaned_features.pt')
wandb.save('/content/protbert_cleaned_features.pt')

wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


['/content/wandb/run-20241023_110932-3fgd4cki/files/content/protbert_cleaned_features.pt']

In [20]:
import logging
import os

# Define log_dir before using it:
log_dir = '/content/drive/My Drive/drug_discovery_datasets/prot_bert_results'  # or your preferred log directory

# Save the model to Google Drive
model_save_path = '/content/drive/My Drive/drug_discovery_datasets/prot_bert_results/prot_bert_trained.pt'
torch.save(model.state_dict(), model_save_path)

# Initialize wandb
wandb.init(project="prot_bert_bfd_project", name="ProtBERT_Training")

# Log the model to wandb
wandb.save(model_save_path)

# Mark the run as complete
wandb.finish()

#print
print(features)

# Set up logging for error tracking
logging.basicConfig(
    filename=os.path.join(log_dir, 'training_errors.log'),
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Function to log error messages
def log_error(message):
    """Logs an error message to the training_errors.log file."""
    logging.error(message)

[array([[ 0.05551019, -0.10461233, -0.03254167, ...,  0.0509166 ,
         0.04319414,  0.10180906],
       [ 0.13895522, -0.04658397,  0.02193595, ...,  0.06942736,
         0.14762989,  0.06503955],
       [ 0.14610627, -0.08092821, -0.12500267, ..., -0.03651211,
         0.02485678,  0.07977513],
       ...,
       [ 0.02349931, -0.01549864, -0.0568524 , ..., -0.0134221 ,
         0.01704358,  0.06431052],
       [ 0.08129995, -0.1092959 , -0.03022885, ...,  0.08717731,
         0.02061544,  0.05156765],
       [ 0.06197345, -0.06417897, -0.02039687, ..., -0.02796473,
         0.08840143,  0.07532729]], dtype=float32), array([[-0.06304298, -0.23687463, -0.0711585 , ..., -0.03852143,
        -0.00322069, -0.0524408 ],
       [ 0.01905619, -0.10517272, -0.02930193, ..., -0.0023861 ,
        -0.09289774,  0.02722568],
       [ 0.07721884, -0.1703196 , -0.13987793, ..., -0.08390196,
         0.03587966, -0.01317166],
       [ 0.00872736, -0.17718184, -0.05856267, ..., -0.09918059,
     